In [4]:
import hashlib
import random
import json
import os

In [70]:
chain_path = os.curdir + '/blocks/'
users = ['Ter Stegen',
              'Semedo', 'Pique', 'Lenglet', 'Alba',
              'Rakitic', 'Busquets', 'De Jong', 
              'Messi', 'Suarez', 'Vidal']
max_amount = 1000000
max_transactions = 100

In [76]:
def fill_structures(amount, sender, receiver):
    userWallets[sender] -= amount
    userWallets[receiver] += amount
    userTransactions[sender].append([-amount, receiver, ''])
    userTransactions[receiver].append([amount, sender, ''])
    

In [40]:
def load_structures_from_files():
    files = os.listdir(chain_path)
    files = sorted([int(i) for i in files])
    for file in files[1:]:
        f = open(chain_path + str(file))
    block = json.load(f)
    for transaction in block['transactions']:
        sender = transaction['sender']
        receiver = transaction['receiver']
        amount = transaction['amount']
        userWallets[sender] -= amount
        userWallets[receiver] += amount
        userTransactions[sender].append([-amount, receiver])
        userTransactions[receiver].append([amount, sender])

In [41]:
def delete_history():
    files = os.listdir(chain_path)
    files = sorted([int(i) for i in files])
    for file in files[1:]:
        os.remove(chain_path + str(file))

In [78]:
userTransactions = {}
for u in users:
    userTransactions[u] = []

In [79]:
userTransactions

{'Ter Stegen': [],
 'Semedo': [],
 'Pique': [],
 'Lenglet': [],
 'Alba': [],
 'Rakitic': [],
 'Busquets': [],
 'De Jong': [],
 'Messi': [],
 'Suarez': [],
 'Vidal': []}

In [80]:
userWallets = {}
for u in users:
    userWallets[u] = 0

In [84]:
userWallets

{'Ter Stegen': 0,
 'Semedo': 7420,
 'Pique': 62434,
 'Lenglet': 137001,
 'Alba': 80676,
 'Rakitic': 10940,
 'Busquets': -67083,
 'De Jong': 59234,
 'Messi': -153269,
 'Suarez': -22629,
 'Vidal': -114724}

In [82]:
#delete_history()

In [83]:
load_structures_from_files()

In [43]:
def get_hash(block_file):
    file = open(chain_path + block_file, 'rb').read()
    return hashlib.sha256(file).hexdigest()

In [86]:
def integrity_check():
    files = os.listdir(chain_path)
    files = sorted([int(i) for i in files])
    for file in files[1:]:
        f = open(chain_path + str(file))
        prev_file_hash = json.load(f)['prev_hash']
        prev_file = str(file - 1)
        prev_file_actual_hash = get_hash(prev_file)
        if prev_file_hash == prev_file_actual_hash:
            result = 'Holistic'
        else:
            result = 'Damaged'
        print('Block {} is {}'.format(prev_file, result))

In [87]:
integrity_check()

Block 0 is Holistic
Block 1 is Holistic
Block 2 is Holistic
Block 3 is Holistic
Block 4 is Holistic
Block 5 is Holistic
Block 6 is Holistic
Block 7 is Holistic
Block 8 is Holistic
Block 9 is Holistic
Block 10 is Holistic


In [45]:
def save_block(transactions):
    files = os.listdir(chain_path)
    files = sorted([int(i) for i in files])

    last_file = 0
    prev_hash = 0
    if len(files) != 0:
        last_file = files[-1]
        prev_hash = get_hash(str(last_file))

    filename = str(last_file + 1)
    
    data = 
    {
        'transactions': transactions,
        'nonce': 0,
        'prev_hash': prev_hash
    }

    with open(chain_path + filename, 'w') as file:
        json.dump(data, file, indent=4)

    while not str(get_hash(filename)).startswith('000'):
        data['nonce'] += random.randint(10, 1000)
        with open(chain_path + filename, 'w') as file:
            json.dump(data, file, indent=4)

In [46]:
def generate_transactions(size):
    transactions = []
    for i in range(size):
        sender = users[random.randint(1, len(users) - 1)]
        receiver = sender
        while receiver == sender:
            receiver = users[random.randint(1, len(users) - 1)]
        wallet = userWallets[sender]
        if wallet > 0:
            amount = random.randint(1, int(wallet / 10))
            fill_structures(amount,sender,receiver)
            transaction = 
            {
                'amount': amount,
                'sender': sender,
                'receiver': receiver
            }
            transactions.append(transaction)

    return transactions

In [69]:
def start_mining(size):
    for i in range(0, size):
        number = random.randint(10, max_transactions)
        transactions = generate_transactions(number)
        save_block(transactions)

In [115]:
def get_wallets():
    for username in users[1:]:
        print(username, ": ", userWallets[username])

In [117]:
get_wallets()

Semedo :  7420
Pique :  62434
Lenglet :  137001
Alba :  80676
Rakitic :  10940
Busquets :  -67083
De Jong :  59234
Messi :  -153269
Suarez :  -22629
Vidal :  -114724


In [98]:
def get_transactions_bigger_than(value):
    for user in users:
        for transaction in userTransactions[user]:
            if(abs(transaction[0]) > value):
                if(transaction[0] > 0): #receiver
                    print('TRANSACTION: ' + user + ' received ' + str(transaction[0]) + ' from ' + str(transaction[1]))
                else:
                    print('TRANSACTION: ' + transaction[1] + ' received ' + str(transaction[0]) + ' from ' + str(user))


In [101]:
#get_transactions_bigger_than(10000)

In [105]:
def get_transactions_less_than(value):
    for user in users:
        for transaction in userTransactions[user]:
            if(abs(transaction[0]) < value):
                if(transaction[0] > 0): #receiver
                    print('TRANSACTION: ' + user + ' received ' + str(abs(transaction[0])) + ' from ' + str(transaction[1]))
                else: #sender
                    print('TRANSACTION: ' + transaction[1] + ' received ' + str(abs(transaction[0])) + ' from ' + str(user))

In [111]:
#get_transactions_less_than(10000)

In [51]:
def initiate_first_transaction():
    transactions = []
    for i in range(len(users) - 1):
        sender = users[0]
        receiver = users[i + 1]
        amount = random.randint(10000, max_amount)
        fill_structures(amount,sender,receiver)
        transaction = 
        {
            'amount': amount,
            'sender': sender,
            'receiver': receiver
        }
        transactions.append(transaction)

    save_block(transactions)

In [107]:
def get_sum_of_transactions_in_block(filename):
    value = 0
    with open(chain_path + str(filename)) as file:
        data = json.load(file)['transactions']
    for transaction in data:
        value += transaction['amount']
    return value

In [110]:
get_sum_of_transactions_in_block(10)

1262892

In [53]:
initiate_mining()

In [54]:
mining_blocks(10)